In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
import sqlite3
import duckdb
di = data_inserts()

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()
# di.insert_datasets()
# di.insert_years()
# di.insert_geo_full()
di.insert_var_full()

In [ ]:
di.conn.execute(
    """
    SELECT * FROM sqlite_db.geo_table;
    """
).df()

In [ ]:
di.conn.execute(
    """
    SELECT * FROM sqlite_db.dataset_table;
    """
).df()

In [ ]:
di.conn.execute(
    """
    SELECT * FROM sqlite_db.year_table;
    """
).df()

In [ ]:
di.conn.execute(
    """
    SELECT * FROM sqlite_db.geo_interm
        WHERE dataset_id=4 AND geo_id=4;
    """
).df()

In [ ]:
url = "http://api.census.gov/data/2017/abscb/variables.json"
results = requests.get(url).json()

In [ ]:
vars_dict = results.get("variables")

df = pl.DataFrame([
    {"name": k, **v}
    for k, v in vars_dict.items()
])
df.sort(["name"])

In [ ]:
candidate = df.select(pl.col("name")).to_series().to_list()

In [ ]:
in_db = di.conn.execute(
    """
    SELECT * FROM sqlite_db.variable_table;
    """
).df()["var_name"].to_list()

In [ ]:
len(candidate)

In [ ]:
[item for item in candidate if item not in in_db]

In [ ]:
len(set(candidate) & set(in_db))